<a href="https://colab.research.google.com/github/vitthal-bhandari/Detecting-Homophobia-and-Transphobia-2.0/blob/master/baselines/English/eng_mBERT_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 15 13:58:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [266]:
!pip install datasets
!pip install transformers
! pip install pyspellchecker
! pip install emoji --upgrade
!sudo apt-get install git-lfs
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html


In [229]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from spellchecker import SpellChecker
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import emoji

In [230]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


# Data wrangling

## Load data

In [127]:
eng_train_dataset = load_dataset("csv", data_files="eng_3_train.tsv", sep="\t")
eng_train = eng_train_dataset["train"]
eng_dev_dataset = load_dataset("csv", data_files="eng_3_dev.tsv", sep="\t")
eng_dev = eng_dev_dataset["train"]
eng_test_dataset = load_dataset("csv", data_files="eng_3_test.tsv", sep="\t")
eng_test = eng_test_dataset["train"]
print(len(eng_train), len(eng_dev), len(eng_test))

Using custom data configuration default-e4c036a87bc4f478
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e4c036a87bc4f478/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-62f8ff8e27b2b629
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-62f8ff8e27b2b629/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-2d520bd614867045
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2d520bd614867045/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

3164 792 999


In [128]:
tam_train_dataset = load_dataset("csv", data_files="tam_3_train.tsv", sep="\t")
tam_train = tam_train_dataset["train"]
tam_dev_dataset = load_dataset("csv", data_files="tam_3_dev.tsv", sep="\t")
tam_dev = tam_dev_dataset["train"]
tam_test_dataset = load_dataset("csv", data_files="tam_3_test.tsv", sep="\t")
tam_test = tam_test_dataset["train"]
print(len(tam_train), len(tam_dev), len(tam_test))

Using custom data configuration default-546649ca8adedccf
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-546649ca8adedccf/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-2ba7caa66734ebc4
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2ba7caa66734ebc4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-4cade1b6a5cce411
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-4cade1b6a5cce411/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

2662 666 833


In [129]:
eng_tam_train_dataset = load_dataset("csv", data_files="eng_tam_3_train.tsv", sep="\t")
eng_tam_train = eng_tam_train_dataset["train"]
eng_tam_dev_dataset = load_dataset("csv", data_files="eng_tam_3_dev.tsv", sep="\t")
eng_tam_dev = eng_tam_dev_dataset["train"]
eng_tam_test_dataset = load_dataset("csv", data_files="eng_tam_3_test.tsv", sep="\t")
eng_tam_test = eng_tam_test_dataset["train"]
print(len(eng_tam_train), len(eng_tam_dev), len(eng_tam_test))

Using custom data configuration default-30656953a6db0651
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-30656953a6db0651/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-5b7322cf9949450c
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5b7322cf9949450c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-61e134c7f20c83a6
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-61e134c7f20c83a6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

3861 966 1207


In [130]:

eng_train_dataset["train"] = eng_train_dataset["train"].remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
eng_train_dataset["train"] = eng_train_dataset["train"].rename_column(original_column_name= 'category', new_column_name= 'label')
eng_train_dataset["train"] = eng_train_dataset["train"].class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e4c036a87bc4f478/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-de1c2eca246dde45.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e4c036a87bc4f478/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-b20c75c585212352.arrow


In [131]:
eng_train = eng_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
eng_train = eng_train.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_train = eng_train.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e4c036a87bc4f478/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-de1c2eca246dde45.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e4c036a87bc4f478/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-b20c75c585212352.arrow


In [132]:
eng_dev = eng_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9'])
eng_dev = eng_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_dev = eng_dev.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-62f8ff8e27b2b629/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-e9acaad71af8c58b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-62f8ff8e27b2b629/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-4d3b579118ccf436.arrow


In [133]:
eng_test = eng_test.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_test = eng_test.rename_column(original_column_name= 'text                        ', new_column_name= 'text')
eng_test = eng_test.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2d520bd614867045/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-d6bd1ba33d6b8312.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2d520bd614867045/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-32d29b3c74dafb07.arrow


In [134]:
eng_test

Dataset({
    features: ['label', 'text'],
    num_rows: 999
})

In [135]:
# remove NA rows in test set
eng_test = eng_test.filter(lambda example, indice: indice < 990, with_indices=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2d520bd614867045/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-4db86def7b4831a7.arrow


In [136]:
eng_test

Dataset({
    features: ['label', 'text'],
    num_rows: 990
})

**Converting Datasets to Datafrane**

In [137]:
eng_train.set_format(type="pandas")
eng_dev.set_format(type="pandas")
eng_test.set_format(type="pandas")

In [138]:
df_eng_train = eng_train[:]
df_eng_dev = eng_dev[:]
df_eng_test = eng_test[:]

df_eng_test

,label,text
0,1,Spr....2016 poitan feel happy with my partner ...
1,1,R u still with ur partner
2,1,excellent movie..no unnecessary drama or scene...
3,1,"For those who speak about culture., pre marita..."
4,1,Best movie and people not understand relations...
...,...,...
985,1,Looks like Karthik took advantage on Varun whe...
986,1,i am really crying pro😢😢😢😭😭😭😭😭😭 ...
987,1,They may be transgender but don't ever forgot ...
988,1,It is their own choice. I support them No doub...


In [139]:
def label_int2str(row):
  return eng_train.features['label'].int2str(row)

df_eng_train['label_name'] = df_eng_train['label'].apply(label_int2str)

In [140]:
def label_int2str2(row):
  return eng_dev.features['label'].int2str(row)

df_eng_dev['label_name'] = df_eng_dev['label'].apply(label_int2str2)

In [141]:
def label_int2str3(row):
  return eng_test.features['label'].int2str(row)

df_eng_test['label_name'] = df_eng_test['label'].apply(label_int2str3)

In [142]:
df_eng_test.head()

,label,text,label_name
0,1,Spr....2016 poitan feel happy with my partner ...,Non-anti-LGBT+ content
1,1,R u still with ur partner,Non-anti-LGBT+ content
2,1,excellent movie..no unnecessary drama or scene...,Non-anti-LGBT+ content
3,1,"For those who speak about culture., pre marita...",Non-anti-LGBT+ content
4,1,Best movie and people not understand relations...,Non-anti-LGBT+ content


## Drop empty rows

In [143]:
df_eng_train = df_eng_train.dropna()
df_eng_dev = df_eng_dev.dropna()
df_eng_test = df_eng_test.dropna()

df_eng_train = df_eng_train.reset_index(drop=True)
df_eng_dev = df_eng_dev.reset_index(drop=True)
df_eng_test = df_eng_test.reset_index(drop=True)

## Remove emojis

In [144]:
df_eng_train['text'] = df_eng_train['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_dev['text'] = df_eng_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_eng_test['text'] = df_eng_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Remove unnecessary puncutation

In [145]:
#keras tokenizer

train_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
train_text_tok.fit_on_texts(df_eng_train['text'])

dev_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
dev_text_tok.fit_on_texts(df_eng_dev['text'])

test_text_tok = Tokenizer(split=' ', lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
test_text_tok.fit_on_texts(df_eng_test['text'])

In [148]:
train_tok = train_text_tok.texts_to_sequences(df_eng_train['text'])

df_eng_train['text'] = df_eng_train.apply(lambda row: train_text_tok.sequences_to_texts([train_tok[row.name]])[0], axis=1)

In [149]:
dev_tok = dev_text_tok.texts_to_sequences(df_eng_dev['text'])

df_eng_dev['text'] = df_eng_dev.apply(lambda row: dev_text_tok.sequences_to_texts([dev_tok[row.name]])[0], axis=1)

In [150]:
test_tok = test_text_tok.texts_to_sequences(df_eng_test['text'])

df_eng_test['text'] = df_eng_test.apply(lambda row: test_text_tok.sequences_to_texts([test_tok[row.name]])[0], axis=1)

## Run spell check

In [196]:
df_eng_train["text"].iloc[0:10]

0                       i support her very smart ponnu
1    priyadharshini kannan  same gender attraction ...
2    bro u name and phone number or mobile number p...
3         experience  thaks bro i love you so much bro
4                     world is becoming bad day by day
5                               plz upload part 2 soon
6    dr venkatesh rajagopalan  sir the mating betwe...
7    i will support lgbt i will support man to man ...
8    these persons can be educated trained for job ...
9    totally moved by the movie superb picturizatio...
Name: text, dtype: object

In [158]:
spell = SpellChecker()

spell.word_frequency.load_words(['lgbt', 'lgbtq', 'lgbtqia+', 'gay', 'lesbian', 'transgender', 'trans'])

In [197]:
def spell_check(row):
  correct_sent = []
  misspelled = spell.unknown(row.split())
  for word in row.split():
    if word in misspelled:
      correct_sent.append(spell.correction(word))
    else:
      correct_sent.append(word)

  return " ".join(correct_sent)

In [198]:
df_eng_train['text'] = df_eng_train['text'].apply(spell_check)
df_eng_dev['text'] = df_eng_dev['text'].apply(spell_check)
df_eng_test['text'] = df_eng_test['text'].apply(spell_check)

In [199]:
df_eng_train["text"].iloc[0:10]

0                       i support her very smart ponzu
1    priyadharshini kanan same gender attraction wi...
2    bro u name and phone number or mobile number p...
3         experience thanks bro i love you so much bro
4                     world is becoming bad day by day
5                               ply upload part 2 soon
6    do venkatesh rajagopalan sir the mating betwee...
7    i will support lgbt i will support man to man ...
8    these persons can be educated trained for job ...
9    totally moved by the movie superb picturizatio...
Name: text, dtype: object

## Reset Dataset format

In [200]:
eng_train.reset_format()
eng_dev.reset_format()
eng_test.reset_format()

# Tokenization

In [201]:
from transformers import AutoTokenizer

model_ckpt = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [202]:
def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [203]:
eng_train = eng_train.filter(lambda example: example['text'] is not None)
eng_dev = eng_dev.filter(lambda example: example['text'] is not None)
eng_test = eng_test.filter(lambda example: example['text'] is not None)
print( len(eng_train), len(eng_dev), len(eng_test) )

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

3162 791 990


In [204]:
#apply tokenizer across all splits in the corpus
eng_train_encoded = eng_train.map(tokenize, batched=True, batch_size=None)
eng_dev_encoded = eng_dev.map(tokenize, batched=True, batch_size=None)
eng_test_encoded = eng_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [205]:
print(eng_train_encoded.column_names)

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


# Approach 1: Feature Extraction

In [206]:
from transformers import AutoModel

model_ckpt = "bert-base-multilingual-cased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [207]:
device

device(type='cuda')

In [208]:
def extract_hidden_states(batch):
  #Place model inputs on GPU
  inputs = {k:v.to(device) for k, v in batch.items() 
  if k in tokenizer.model_input_names}

  #Extract last hidden states
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  
  #Return vector for [CLS] token
  return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [209]:
eng_train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [210]:
eng_dev_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [211]:
eng_train_hidden = eng_train_encoded.map(extract_hidden_states, batched=True)
eng_dev_hidden = eng_dev_encoded.map(extract_hidden_states, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [212]:
eng_train_hidden.column_names

['label',
 'text',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [213]:
X_eng_train = np.array(eng_train_hidden['hidden_state'])
X_eng_dev = np.array(eng_dev_hidden['hidden_state'])

Y_eng_train = np.array(eng_train_hidden['label'])
Y_eng_dev = np.array(eng_dev_hidden['label'])

print( X_eng_train.shape, X_eng_dev.shape, Y_eng_train.shape, Y_eng_dev.shape)

(3162, 768) (791, 768) (3162,) (791,)


## Training an LR Classifier

In [214]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter = 3000)
lr_clf.fit(X_eng_train, Y_eng_train)

LogisticRegression(max_iter=3000)

In [215]:
lr_clf.score(X_eng_dev, Y_eng_dev)

0.922882427307206

In [216]:
#Evaluating a dummy classifier for comparison which always returns the majority class
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_eng_train, Y_eng_train)
dummy_clf.score(X_eng_dev, Y_eng_dev)

0.9241466498103666

In [217]:
Y_eng_preds = lr_clf.predict(X_eng_dev)

In [218]:
tf.math.confusion_matrix(Y_eng_dev, Y_eng_preds)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 10,  48,   0],
       [ 11, 720,   0],
       [  0,   2,   0]], dtype=int32)>

In [219]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_dev, Y_eng_preds, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.48      0.17      0.25        58
Non-anti LGBTQIA+ content       0.94      0.98      0.96       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.92       791
                macro avg       0.47      0.39      0.40       791
             weighted avg       0.90      0.92      0.91       791



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Approach 2: Fine Tuning

In [220]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [221]:
#Define performance metrics

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "macro")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [222]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(eng_train_encoded) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 5,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [223]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_encoded,
    eval_dataset = eng_dev_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.250300,0.281237,0.924147,0.320193
2,0.172500,0.211617,0.926675,0.371069
3,0.130900,0.204564,0.936789,0.469464
4,0.084100,0.268927,0.927939,0.459644
5,0.047700,0.309815,0.932996,0.453514


TrainOutput(global_step=495, training_loss=0.13596899048848585, metrics={'train_runtime': 434.918, 'train_samples_per_second': 36.352, 'train_steps_per_second': 1.138, 'total_flos': 2079911567139840.0, 'train_loss': 0.13596899048848585, 'epoch': 5.0})

In [224]:
preds_output = trainer.predict(eng_dev_encoded)

In [225]:
preds_output.metrics

{'test_accuracy': 0.9329962073324906,
 'test_f1': 0.45351431566057515,
 'test_loss': 0.30981457233428955,
 'test_runtime': 7.7159,
 'test_samples_per_second': 102.516,
 'test_steps_per_second': 3.24}

In [226]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [227]:
tf.math.confusion_matrix(Y_eng_dev, y_preds)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 17,  41,   0],
       [ 10, 721,   0],
       [  1,   1,   0]], dtype=int32)>

In [228]:
print(classification_report(Y_eng_dev, y_preds, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.61      0.29      0.40        58
Non-anti LGBTQIA+ content       0.94      0.99      0.97       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.93       791
                macro avg       0.52      0.43      0.45       791
             weighted avg       0.92      0.93      0.92       791



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# EDA

**Saving dataset in a csv format classwise separately**

In [231]:
eng_train.set_format(type="pandas")
eng_dev.set_format(type="pandas")
eng_test.set_format(type="pandas")

In [232]:
df_eng_train = eng_train[:]
df_eng_dev = eng_dev[:]
df_eng_test = eng_test[:]

df_eng_train

,label,text
0,1,"I support her, very smart ponnu"
1,0,priyadharshini kannan same gender attraction ...
2,1,Bro u name and phone number (or)mobile number ...
3,1,experience Thaks bro I love you so much bro
4,1,world is becoming bad day by day....
...,...,...
3157,1,That Akka is really good soul
3158,1,Good madam.if i get any oppertunity in future ...
3159,1,judgment is based on scientific facts
3160,1,Respect to allll🙏🙏🙏


In [233]:
df_homophobic=df_eng_train[df_eng_train.label == 0]
df_homophobic.head()

,label,text
1,0,priyadharshini kannan same gender attraction ...
22,0,Really excellent movie....I feel that no one a...
31,0,Madam read bible
94,0,it's due to abnormalities present sexual dimor...
96,0,i think u too gay....thats y u telling with t...


In [234]:
df_transphobic=df_eng_train[df_eng_train.label == 2]
df_transphobic.head()

,label,text
401,2,Sex the 9.
1090,2,Magalakshmi Mukunthan Ella transgalayum konnud...
1340,2,Today also one transgender scolded me for not ...
1418,2,Rajeswari Rajagopalan ; Sister....last week fo...
2330,2,Hey seriously I thought She was a Transgender


**Applying EDA to homophobia and transphobia classes**

In [236]:
import nltk

nltk.download('wordnet')
!git clone https://github.com/jasonwei20/eda_nlp.git
%cd eda_nlp/code

!ls

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Cloning into 'eda_nlp'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 396 (delta 6), reused 10 (delta 4), pack-reused 379
Receiving objects: 100% (396/396), 20.42 MiB | 22.41 MiB/s, done.
Resolving deltas: 100% (189/189), done.
/content/eda_nlp/code
augment.py  eda.py  __pycache__


In [248]:
df_homophobic.to_csv('homophobic.txt', sep="\t", header=False, index=False)
df_transphobic.to_csv('transphobic.txt', sep="\t", header=False, index=False)

In [249]:
! python augment.py --input=transphobic.txt --num_aug=32

generated augmented sentences with eda for transphobic.txt to eda_transphobic.txt with num_aug=32


In [250]:
! python augment.py --input=homophobic.txt --num_aug=16

generated augmented sentences with eda for homophobic.txt to eda_homophobic.txt with num_aug=16


**Combine augmented data with existing data**

In [251]:
df_homophobic_aug=pd.read_csv('eda_homophobic.txt', sep="\t", names=["label", "text"])
df_transphobic_aug=pd.read_csv('eda_transphobic.txt', sep="\t", names=["label", "text"])

In [252]:
df_homophobic_aug.describe()

,label
count,2669.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [253]:
df_ally=df_eng_train[df_eng_train.label == 1]
df_ally.describe()

,label
count,2999.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [254]:
frames = [df_ally, df_homophobic, df_transphobic, df_homophobic_aug, df_transphobic_aug]
df_aug = pd.concat(frames)

In [255]:
# re shuffling dataframe

df_aug_shuffled = df_aug.sample(frac=1).reset_index(drop=True)
df_aug_shuffled.info

<bound method DataFrame.info of       label                                               text
0         1      Its a big gift to lgbt community thanks a lot
1         0  momosexual are more affected cupids disease wi...
2         2  every transenter says they are doing prostitui...
3         0                                          ples them
4         0  govind svi to i understand ur problems but leg...
...     ...                                                ...
6024      0                            to god turn wicked ways
6025      0  bible says corinthians divinity know ye not th...
6026      1                               Thank you brother :)
6027      1                                   Really  😭😭😭😭😭😭😭😭
6028      0  nice movie but death not possible india relati...

[6029 rows x 2 columns]>

In [256]:
df_aug_shuffled['label'].value_counts()

1    2999
0    2826
2     204
Name: label, dtype: int64

In [257]:
df_aug_shuffled.to_csv('augmented_data.csv')

# Approach 3: Fine Tuning with EDA

**We convert augmented dataset back to a Datasets object and run tokenization again, before fine-tuning it**

In [258]:
from datasets import Dataset
eng_train_aug = Dataset.from_pandas(df = df_aug_shuffled)

In [259]:
eng_train_aug

Dataset({
    features: ['label', 'text'],
    num_rows: 6029
})

In [260]:
eng_train_aug_encoded = eng_train_aug.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [261]:
eng_train_aug_encoded.column_names

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

In [262]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(eng_train_encoded) // batch_size
model_name = f"Homophobia-Transphobia-v2-mBERT-EDA"
training_args = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 5,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    push_to_hub=True,
    log_level = "error"
)

In [267]:

trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_aug_encoded,
    eval_dataset = eng_dev_encoded,
    tokenizer = tokenizer
)

trainer.train()

Cloning https://huggingface.co/bitsanlp/Homophobia-Transphobia-v2-mBERT-EDA into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.169900,0.412520,0.922882,0.463358
2,0.038700,0.465798,0.922882,0.368896
3,0.014800,0.525037,0.935525,0.437593
4,0.000500,0.533559,0.931732,0.453068
5,0.001600,0.540100,0.931732,0.449815


TrainOutput(global_step=945, training_loss=0.03487075007072202, metrics={'train_runtime': 816.8419, 'train_samples_per_second': 36.904, 'train_steps_per_second': 1.157, 'total_flos': 3965776988705280.0, 'train_loss': 0.03487075007072202, 'epoch': 5.0})

In [268]:
preds_output = trainer.predict(eng_dev_encoded)

In [269]:
y_preds_aug = np.argmax(preds_output.predictions, axis=1)

In [270]:
tf.math.confusion_matrix(Y_eng_dev, y_preds_aug)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 16,  42,   0],
       [  9, 721,   1],
       [  0,   2,   0]], dtype=int32)>

In [271]:
print(classification_report(Y_eng_dev, y_preds_aug, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.64      0.28      0.39        58
Non-anti LGBTQIA+ content       0.94      0.99      0.96       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.93       791
                macro avg       0.53      0.42      0.45       791
             weighted avg       0.92      0.93      0.92       791

